![Henry Logo](https://www.soyhenry.com/_next/static/media/HenryLogo.bb57fd6f.svg)

# Modulo 2 (Extension): Diseno de Base de Datos Vectorial con Batman

## Objetivo
Construir una base vectorial desde cero, justificando decisiones de diseno como lo hariamos en un curso de AI Engineering aplicado.

## Resultado esperado
- Dataset estructurado y chunking reproducible.
- Indexacion vectorial en ChromaDB.
- Recuperacion semantica con metadata util para filtros y trazabilidad.

## Blueprint de diseno (resumen ejecutivo)

1. **Modelado de documentos**: cada comic se conserva con `id`, `personaje`, `arco`, `tema`, `titulo`, `contenido`.
2. **Chunking**: tamano intermedio (800 chars) con overlap (120) para balancear contexto vs precision.
3. **Embeddings**: `text-embedding-3-small` para costo/calidad en produccion educativa.
4. **Indice**: ChromaDB persistente para consultas, inspeccion y evaluacion reproducible.
5. **Observabilidad**: estadisticas de chunks + inspeccion de retrieval para detectar drift semantico.

## Justificacion teorica de las decisiones de diseno

### Chunk size = 800 caracteres

El tamano de chunk es una de las decisiones mas impactantes en el diseno de una base vectorial. Consideremos los trade-offs:

| Chunk size | Ventaja | Desventaja |
|---|---|---|
| **Pequeno** (200-400 chars) | Alta precision: cada chunk es semanticamente coherente | Pierde contexto narrativo; requiere mas chunks para cubrir una idea completa |
| **Medio** (600-1000 chars) | Buen balance: contexto suficiente para respuestas coherentes | Puede mezclar ideas en fronteras de parrafo |
| **Grande** (1500+ chars) | Contexto rico para respuestas complejas | Dilution semantica: el embedding promedia temas heterogeneos, reduce recall |

Elegimos **800 chars** (~160 tokens) porque:
1. Es significativamente menor que el limite del modelo de embedding (`text-embedding-3-small` soporta hasta 8191 tokens ≈ 5000+ chars), lo cual evita truncamiento.
2. Para textos narrativos como comics, 800 chars captura entre 1-2 parrafos completos — suficiente para que el LLM construya una respuesta coherente sin necesitar multiples chunks del mismo pasaje.
3. Empiricamente, chunks de este tamano producen un buen ratio precision/recall en retrieval: lo suficientemente especificos para matchear consultas focalizadas, pero con contexto suficiente para no ser fragmentos incoherentes.

### Chunk overlap = 120 caracteres (15% del chunk_size)

El overlap controla cuanto contenido se repite entre chunks consecutivos. Su funcion es **preservar continuidad semantica** en las fronteras de fragmentacion:

- **Overlap = 0**: riesgo de cortar una idea a la mitad sin que ninguno de los dos chunks la represente completa. Una consulta sobre esa idea no matchearia con ningun chunk.
- **Overlap demasiado alto (>30%)**: genera duplicacion significativa que infla el indice sin beneficio proporcional, aumenta costos de embedding, y puede sesgar el retrieval hacia contenido repetido.
- **15% (~120 chars)**: preserva ~1-2 oraciones de contexto entre chunks. Suficiente para que ideas en fronteras de parrafo aparezcan en al menos un chunk con contexto adecuado.

### Modelo de embedding: `text-embedding-3-small`

OpenAI ofrece dos variantes de `text-embedding-3`:
- `text-embedding-3-small`: 1536 dimensiones, ~5x mas barato que large.
- `text-embedding-3-large`: 3072 dimensiones, mayor capacidad discriminativa.

Para textos narrativos como los de este laboratorio, `small` ofrece calidad de retrieval comparable a `large` a una fraccion del costo. La diferencia se nota mas en dominios altamente tecnicos con vocabulario especializado. En un contexto educativo y de prototipado, `small` es la eleccion correcta.

### Cosine similarity y HNSW en ChromaDB

ChromaDB usa por defecto **cosine similarity** como metrica de distancia en su indice HNSW (Hierarchical Navigable Small World). Implicaciones:
- Los vectores se comparan por **direccion**, no por magnitud. Dos textos sobre el mismo tema tendran vectores con angulo pequeno (alta similaridad) sin importar su longitud.
- HNSW es un grafo de proximidad que permite busqueda aproximada en O(log n), haciendo viable el retrieval en tiempo real incluso con miles de chunks.
- La distancia coseno retornada por ChromaDB esta en rango [0, 2], donde 0 = identico y 2 = opuesto. Valores tipicos para chunks relevantes: 0.2–0.6.

In [1]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

ROOT = Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from scripts.common import load_comic_records, embed_documents
from scripts.vector_store_lab import build_index_from_json

DATA_PATH = ROOT / 'data' / 'batman_comics.json'
PERSIST_DIR = ROOT / 'outputs' / 'chroma_batman_design'
PERSIST_DIR.mkdir(parents=True, exist_ok=True)
OUTPUTS_DIR = ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

print(f'Notebook root: {ROOT}')
print(f'Data path: {DATA_PATH}')

/Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Notebook root: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/batman_vector_db_orchestration
Data path: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/batman_vector_db_orchestration/data/batman_comics.json


In [2]:
records = load_comic_records(DATA_PATH)
records_df = pd.DataFrame(records)
print(f'Total records: {len(records_df)}')
records_df[['id', 'personaje', 'arco', 'tema']].head(8)

Total records: 12


,id,personaje,arco,tema
0,batman_01,batman,Year One,origen
1,batman_02,batman,The Killing Joke,villanos
2,batman_03,batman,The Dark Knight Returns,legado
3,batman_04,batman,Court of Owls,conspiracion
4,batman_05,batman,Knightfall,derrota
5,batman_06,batman,Hush,misterio
6,batman_07,batman,Catwoman,relaciones
7,batman_08,batman,Liga de la Justicia,equipo


## Discusion tecnica: decisiones de arquitectura

| Componente | Decision | Justificacion aplicada |
|---|---|---|
| Unidad de indexacion | Chunk por fragmento narrativo | Reduce dilution semantica en documentos largos |
| Metadata minima | `personaje`, `arco`, `tema`, `titulo`, `source_id`, `chunk_index` | Permite filtros, auditoria y debugging de retrieval |
| Store | ChromaDB persistente | Simplicidad operativa para laboratorio y prototipos |
| Embeddings | `text-embedding-3-small` | Buen trade-off costo/recall para contenido textual denso |
| Eval inicial | Muestra de queries + distancias | Detecta huecos de cobertura antes del tuning fino |

In [3]:
db, chunks, index_stats, chunk_stats = build_index_from_json(
    json_path=DATA_PATH,
    persist_dir=PERSIST_DIR,
    collection_name='batman_design_lab',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

print('Index stats:')
print(index_stats)
print('\nChunk stats:')
print(chunk_stats)

chunks_df = pd.DataFrame([
    {
        'id': chunk['id'],
        'chars': len(chunk['text']),
        'tema': chunk['metadata']['tema'],
        'arco': chunk['metadata']['arco'],
    }
    for chunk in chunks
])
chunks_df.head()

Index stats:
{'collection': 'batman_design_lab', 'indexed_chunks': 38, 'embedding_provider': 'local:hash-embedding'}

Chunk stats:
{'chunk_count': 38, 'unique_sources': 12, 'avg_chars_per_chunk': 682.5, 'max_chars_per_chunk': 792, 'themes_distribution': {'origen': 3, 'villanos': 3, 'legado': 3, 'conspiracion': 3, 'derrota': 3, 'misterio': 3, 'relaciones': 3, 'equipo': 3, 'escenario': 3, 'mentoria': 4, 'recursos': 4, 'filosofia': 3}, 'hero_distribution': {'batman': 38}}


,id,chars,tema,arco
0,batman_01_chunk_000,781,origen,Year One
1,batman_01_chunk_001,713,origen,Year One
2,batman_01_chunk_002,648,origen,Year One
3,batman_02_chunk_000,715,villanos,The Killing Joke
4,batman_02_chunk_001,723,villanos,The Killing Joke


In [4]:
sample_queries = [
    'Como inicia Batman su carrera en Gotham?',
    'Que estrategia usa Bane para derrotar a Batman en Knightfall?',
    'Que rol cumple Batman dentro de la Liga de la Justicia?',
]

rows = []
for query in sample_queries:
    results, provider = db.query(query_text=query, n_results=3, embedding_model='text-embedding-3-small')
    for item in results:
        rows.append({
            'query': query,
            'rank': item['rank'],
            'distance': round(item['distance'], 4),
            'tema': item['metadata'].get('tema'),
            'arco': item['metadata'].get('arco'),
            'snippet': item['text'][:160] + '...',
            'retrieval_provider': provider,
        })

retrieval_df = pd.DataFrame(rows)
retrieval_df

,query,rank,distance,tema,arco,snippet,retrieval_provider
0,Como inicia Batman su carrera en Gotham?,1,0.6458,relaciones,Catwoman,La relacion entre Batman y Catwoman es la mas ...,local:hash-embedding
1,Como inicia Batman su carrera en Gotham?,2,0.7295,derrota,Knightfall,Bane no es un villano comun. Nacido en la pris...,local:hash-embedding
2,Como inicia Batman su carrera en Gotham?,3,0.7316,legado,The Dark Knight Returns,Es el ano 1986 en la cronologia de Frank Mille...,local:hash-embedding
3,Que estrategia usa Bane para derrotar a Batman...,1,0.5605,relaciones,Catwoman,". En Hush, Jeph Loeb los hizo pareja oficial. ...",local:hash-embedding
4,Que estrategia usa Bane para derrotar a Batman...,2,0.6406,relaciones,Catwoman,". Ella roba, pero solo a los ricos y corruptos...",local:hash-embedding
5,Que estrategia usa Bane para derrotar a Batman...,3,0.6783,relaciones,Catwoman,La relacion entre Batman y Catwoman es la mas ...,local:hash-embedding
6,Que rol cumple Batman dentro de la Liga de la ...,1,0.4238,equipo,Liga de la Justicia,. Si alguno de ustedes se vuelve loco o es con...,local:hash-embedding
7,Que rol cumple Batman dentro de la Liga de la ...,2,0.4474,recursos,Tecnologia,". El peto es kevlar trenzado con titanio, capa...",local:hash-embedding
8,Que rol cumple Batman dentro de la Liga de la ...,3,0.4896,mentoria,Los Robins,". Donde Dick era luz, Jason era fuego. Impulsi...",local:hash-embedding


In [5]:
sample_texts = [chunk['text'] for chunk in chunks[:60]]
sample_meta = [chunk['metadata'] for chunk in chunks[:60]]
vectors, embedding_provider = embed_documents(sample_texts, model='text-embedding-3-small')

pca = PCA(n_components=2, random_state=42)
projection = pca.fit_transform(vectors)

proj_df = pd.DataFrame({
    'x': projection[:, 0],
    'y': projection[:, 1],
    'tema': [meta['tema'] for meta in sample_meta],
})

fig, ax = plt.subplots(figsize=(8, 5))
for tema, subset in proj_df.groupby('tema'):
    ax.scatter(subset['x'], subset['y'], label=tema, alpha=0.75)

ax.set_title(f'Proyeccion de embeddings (provider={embedding_provider})')
ax.set_xlabel('PCA-1')
ax.set_ylabel('PCA-2')
ax.grid(alpha=0.3)
ax.legend(loc='best', fontsize=8)

plot_path = OUTPUTS_DIR / 'vector_db_design_projection.png'
fig.tight_layout()
fig.savefig(plot_path, dpi=150)
plt.close(fig)

print(f'Saved: {plot_path}')

Saved: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/batman_vector_db_orchestration/outputs/vector_db_design_projection.png


## Cierre didactico

### Conceptos clave de esta notebook

- **Chunking no es trivial**: el tamano y overlap de fragmentos afectan directamente la calidad del retrieval. No hay valores universales — dependen del dominio, la longitud de los documentos y el tipo de consultas esperadas. Los valores elegidos (800/120) son un punto de partida razonable que debe validarse empiricamente.
- **Metadata es infraestructura de observabilidad**: campos como `personaje`, `arco`, `tema` y `source_id` no son decorativos — permiten filtrar, auditar y debuggear el comportamiento del retrieval en produccion.
- **El embedding model importa menos de lo que se piensa** (dentro de la misma generacion): para la mayoria de casos de uso con texto en espanol/ingles, `text-embedding-3-small` produce resultados comparables a `large` a una fraccion del costo.
- **La visualizacion PCA de embeddings** es una herramienta de diagnostico, no de evaluacion. Si los clusters tematicos son visibles en 2D, el modelo de embedding esta capturando estructura semantica util. Si no, puede indicar que los chunks son demasiado homogeneos o el chunking es demasiado agresivo.

### Conexion con la siguiente notebook

Con la base vectorial operativa y auditada, en la siguiente notebook la usamos para comparar **Vanilla RAG** vs **Agentic RAG** con metricas cuantitativas (groundedness, latencia) y graficas comparativas. La pregunta central sera: *cuando justifica la complejidad adicional de un pipeline agentico?*